In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from src.parsers import mimic, hirid
from src.modeling import discovery, plots, querier
from src.utils import constants

In [4]:
import sys
import win32com.client
import os
import pandas as pd

def setup_io_config(root_path):
    """
    Input - Output config. Add dataset paths
    :root_path -> Repo path which contains 'data' and 'res' folders
    """

    # MIMIC
    is_shortcut = True if "data.lnk" in os.listdir(root_path) else False 
    
    if (is_shortcut):
        path_shortcut =  os.path.join(root_path, "data.lnk")
        shell = win32com.client.Dispatch("WScript.Shell")
        data = shell.CreateShortCut(path_shortcut).Targetpath
    else:
        data = f"{root_path}\\data"
    
    res = f"{root_path}\\results"

    # HIRID
    raw_path = f'{root_path}/data/hirid-a-high-time-resolution-icu-dataset-1.1.1/raw_stage/'
    res_path = f'{root_path}/data/hirid-a-high-time-resolution-icu-dataset-1.1.1'
    
    return data, res, raw_path, res_path

def setup_stratification_config():
    gender="MF"
    age_b=40
    age_a=80 
    ethnicity="WHITE" 
    lab_mapping= constants.LAB_MAPPING
    before_windows = [(0,12)]
    after_windows = [(0,12)]
    return gender, age_a, age_b, ethnicity, lab_mapping, before_windows, after_windows

In [5]:
# IO Config
root_path ="C:\\Users\\danco\\My Drive\\Master\\Datasets\\MIMIC iii"
#root_path = "/Users/pavan/Library/CloudStorage/GoogleDrive-f20190038@hyderabad.bits-pilani.ac.in/My Drive/TAU/Code/DrugLab"
data, res, raw_path, res_path = setup_io_config(root_path=root_path)

# Stratification Config
gender, age_a, age_b, ethnicity, lab_mapping, before_windows, after_windows = setup_stratification_config()

In [7]:
# MIMIC
mimic_parser = mimic.MIMICParser(data=data, res=res, gender=gender, age_b=age_b, age_a=age_a, ethnicity=ethnicity, lab_mapping=lab_mapping)
m_med1, m_med2, m_labs = mimic_parser.parse()
## Querier
mimic_data_querier = querier.DatasetQuerier(
    data = data,
    res = res,
    gender=gender, 
    age_b=age_b, 
    age_a=age_a, 
    ethnicity=ethnicity, 
    lab_mapping=lab_mapping
)
m_final_lab_med_data = mimic_data_querier.generate_med_lab_data(m_labs, m_med1, m_med2, before_windows, after_windows)

C:\Users\danco\My Drive\Master\Projects\DrugLab\src\parsers\mimic.py:93: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  med2 = pd.read_csv(os.path.join(self.data, "mimiciii/1.4/preprocessed", "med2_vectorized.csv")) if use_med_vect and os.path.exists(os.path.join(self.data, "mimiciii/1.4/preprocessed", "med2_vectorized.csv")) else self.generate_med2_vect()
C:\Users\danco\My Drive\Master\Projects\DrugLab\src\modeling\querier.py:123: FutureWarning: Passing 'suffixes' which cause duplicate columns {'after_trends_(0, 12)_x', 'after_mean_(0, 12)_x', 'before_trends_(0, 12)_x', 'before_mean_(0, 12)_x'} in the result is deprecated and will raise a MergeError in a future version.
  col_vals[i] = pd.merge(col_vals[i-1], col_vals[i], how="outer", on=list(t_med1.columns)+["LAB_ITEMID"])
C:\Users\danco\My Drive\Master\Projects\DrugLab\src\modeling\querier.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [82]:
plotter = plots.ClinicalPlotAnalysis(
    data = data,
    res = res,
    gender=gender, 
    age_b=age_b, 
    age_a=age_a, 
    ethnicity=ethnicity, 
    lab_mapping=lab_mapping
)
m_corrs_data_df = plotter.plot(m_final_lab_med_data, m_labs, before_windows=before_windows, after_windows=after_windows)
#m_final_lab_med_data[1]
#m_final_lab_med_data[1].groupby(['SUBJECT_ID'])['ITEMID'].count()

AttributeError: 'NoneType' object has no attribute 'groupby'

In [84]:
{k:list(v.keys()) for k, v in constants.LAB_MAPPING.items()}

{'ALT': ['Alanine aminotransferase'],
 'ANA': ['Fraction inspired oxygen'],
 'AST': ['Asparate aminotransferase'],
 'Hemoglobin': ['Hemoglobin percent',
  'Hemoglobin C',
  'Hemoglobin F',
  'Hemoglobin A2',
  'Hemoglobin'],
 'INR': ['Prothrombin time INR'],
 'bilirubin': ['Bilirubin'],
 'calcium': ['Calcium', 'Calcium ionized'],
 'creatinine': ['Creatinine', 'Creatinine ascites'],
 'glucose': ['Glucose'],
 'lactic acid': ['Lactic acid'],
 'magnesium': ['Magnesium', 'Magnesium, Urine'],
 'platelets': ['Platelets', 'Large Platelets'],
 'potassium': ['Potassium'],
 'sodium': ['Sodium'],
 'Uric acid': ['Uric Acid', 'Uric Acid, Urine'],
 'B12': ['Vitamin B12'],
 'prolactin': ['Prolactin'],
 'Amylase': ['Amylase'],
 'Lipase': ['Lipase'],
 'Aptt': ['PTT'],
 'Hematocrit': ['Hematocrit'],
 'Red blood cell': ['Red blood cell'],
 'Albumin': ['Albumin'],
 'Magnesium': ['Magnesium'],
 'CPK': []}

In [15]:
#os.mkdir('bb')
#stratify_prefix = f"{age_b}-{age_a}_{gender}_{ethnicity}"
res = f"{root_path}\\results"

ccc = os.path.join(res,f"before_after_windows_main_med_lab_first_val_{stratify_prefix}_doc_eval_new_win.csv")
#res, 
ccc
#d_m_l_doc.to_csv(ccc)
#pd.DataFrame.from_dict(res_dict_mapping_med)

'C:\\Users\\danco\\My Drive\\Master\\Datasets\\MIMIC iii\\results\\before_after_windows_main_med_lab_first_val_40-80_MF_WHITE_doc_eval_new_win.csv'

In [12]:
import pandas as pd

labs = pd.read_csv(os.path.join(data, "mimiciii/1.4/preprocessed", "lab_patient_data_mimic_extract_2.csv")) 

In [27]:
d_m_l_doc = pd.read_csv(os.path.join(data, "mimiciii", "1.4","preprocessed", "mapping_med_itemid_doc.csv")).drop(columns=["Unnamed: 0"])
dict_d_m_l = d_m_l_doc.to_dict("records")
res_dict_mapping_med = {v:k["Medication"] for k in dict_d_m_l for v in [int(id) for id in k["ITEMID_with_manual"][1:-1].split(",") if id != '']}

In [28]:
import pandas as pd
gender="MF"
age_b=40
age_a=80 
ethnicity="WHITE" 
lab_mapping= constants.LAB_MAPPING
before_windows = [(0,12)]
after_windows = [(0,12)]
use_med_vect = True

med1 = pd.read_csv(os.path.join(data, "mimiciii/1.4/preprocessed", "med1_vectorized.csv")) if use_med_vect and os.path.exists(os.path.join(
    data, "mimiciii/1.4/preprocessed", "med1_vectorized.csv")) else generate_med1_vect()
h_adm_1 = med1.sort_values(["HADM_ID", "STARTTIME"]).groupby("SUBJECT_ID").nth(0)["HADM_ID"].to_list()
med1 = med1[med1.HADM_ID.isin(h_adm_1)]
med1 = med1.drop(columns=["Unnamed: 0"])
med1 = med1[med1["AGE"]>=age_b]
med1 = med1[med1["AGE"]<=age_a]
med1 = med1[med1["GENDER"]==gender] if gender != "MF" else med1
med1 = med1[med1["ETHNICITY"]==ethnicity]
med1["MIMICExtractLabel"] = med1.apply(lambda r: res_dict_mapping_med[r["ITEMID"]] if r["ITEMID"] in res_dict_mapping_med else r["LABEL"], axis=1)
med1["STARTTIME"] = pd.to_datetime(med1["STARTTIME"])
med1["ENDTIME"] = pd.to_datetime(med1["ENDTIME"])
med1["ADMITTIME"] = pd.to_datetime(med1["ADMITTIME"])
med1["MedTimeFromAdmit"] = med1["ENDTIME"]-med1["ADMITTIME"]
med1["hours_in"] = med1["MedTimeFromAdmit"].dt.total_seconds()/3600

In [77]:
#use_med_vect = True
med1_old = pd.read_csv(os.path.join(data, "mimiciii/1.4/preprocessed", "med1_vectorized.csv"))
med1

,SUBJECT_ID,ITEMID,HADM_ID,ROW_ID_x,ICUSTAY_ID,STARTTIME,ENDTIME,AMOUNT,AMOUNTUOM,RATE,...,MedTimeFromAdmit,ROW_ID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
45596,41624,226372,178751,1538868,277546.0,2162-05-06 11:00:00,2162-05-06 11:01:00,600.000000,ml,NaN,...,0 days 00:01:00,14457,OR Cell Saver Intake,OR Cell Saver Intake,metavision,inputevents_mv,Blood Products/Colloids,mL,Solution,NaN
103469,53729,226364,140334,1857294,289288.0,2150-09-30 13:00:00,2150-09-30 13:01:00,4000.000000,ml,NaN,...,0 days 00:01:00,14450,OR Crystalloid Intake,OR Crystalloid Intake,metavision,inputevents_mv,Fluids/Intake,mL,Solution,NaN
88606,50735,226363,175220,3253371,239857.0,2190-02-23 19:17:00,2190-02-23 19:18:00,600.000000,ml,NaN,...,0 days 00:01:00,14449,Cath Lab Intake,Cath Lab Intake,metavision,inputevents_mv,Fluids/Intake,mL,Solution,NaN
82501,49469,226361,160152,189972,208590.0,2111-11-06 22:16:00,2111-11-06 22:17:00,1000.000000,ml,NaN,...,0 days 00:01:00,14447,Pre-Admission Intake,Pre-Admission Intake,metavision,inputevents_mv,Fluids/Intake,mL,Solution,NaN
195772,72555,226361,194577,2690710,227291.0,2144-08-09 06:51:00,2144-08-09 06:52:00,2000.000000,ml,NaN,...,0 days 00:01:00,14447,Pre-Admission Intake,Pre-Admission Intake,metavision,inputevents_mv,Fluids/Intake,mL,Solution,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291681,93321,225797,115396,3599579,248745.0,2128-09-14 17:58:00,2128-09-14 17:59:00,100.000000,ml,NaN,...,173 days 18:58:00,13981,Free Water,Free Water,metavision,inputevents_mv,Fluids/Intake,mL,Solution,NaN
291702,93321,225930,115396,3599570,248745.0,2128-09-15 08:22:00,2128-09-15 11:12:00,84.919287,ml,29.971513,...,174 days 12:11:00,14367,Peptamen 1.5 (Full),Peptamen 1.5 (Full),metavision,inputevents_mv,Nutrition - Enteral,mL,Solution,NaN
291701,93321,225917,115396,3599565,248745.0,2128-09-14 17:53:00,2128-09-15 17:54:00,2497.699998,ml,103.998612,...,174 days 18:53:00,14355,TPN without Lipids,TPN without Lipids,metavision,inputevents_mv,Nutrition - Parenteral,mL,Solution,NaN
291693,93321,225883,115396,3599574,248745.0,2128-09-16 00:30:00,2128-09-16 00:31:00,1.000000,dose,NaN,...,175 days 01:30:00,14041,Meropenem,Meropenem,metavision,inputevents_mv,Antibiotics,dose,Solution,NaN


In [73]:
#med1
#time_diff = (med1.ENDTIME - med1.STARTTIME).dt.total_seconds()/3600
#(time_diff < 2).value_counts()
med1 = m_p_df.groupby(["HADM_ID", "ITEMID"]).nth(0).reset_index()
med1 = med1.sort_values("MedTimeFromAdmit")
med1 = med1[med1["MedTimeFromAdmit"].dt.total_seconds()>0]
med1 = med1.sort_values(by=["ADMITTIME"]).groupby(["SUBJECT_ID", "ITEMID"]).nth(0).reset_index().sort_values(by=["MedTimeFromAdmit"])

med1.groupby('LABEL')['SUBJECT_ID'].count()

LABEL
ACD-A Citrate (1000ml)      17205
ACD-A Citrate (500ml)         316
Abciximab (Reopro)              7
Acetaminophen-IV             4523
Acyclovir                    4258
                            ...  
Vitamin K (Phytonadione)      325
Vivonex (1/2)                  42
Vivonex (3/4)                  12
Vivonex (Full)               2082
Voriconazole                  873
Name: SUBJECT_ID, Length: 278, dtype: int64

In [76]:

med1.groupby('LABEL')['SUBJECT_ID'].count().sort_values(ascending=False)

LABEL
NaCl 0.9%              15888
Dextrose 5%            13861
PO Intake              13360
Solution               11192
Magnesium Sulfate       9276
                       ...  
Isosource 1.5 (2/3)        1
Ensure (1/4)               1
Ensure (3/4)               1
Pulmocare (1/4)            1
Quinine                    1
Name: SUBJECT_ID, Length: 278, dtype: int64

In [80]:
#final_mapping_lab_itemids = {v2:k1 for k, v in constants.LAB_MAPPING.items() for k1, v1 in v.items() for v2 in v1}
#final_itemids_list = list(final_mapping_lab_itemids.keys())
final_itemids_list

[50861,
 769,
 220644,
 189,
 50878,
 770,
 220587,
 50852,
 51224,
 51225,
 51223,
 814,
 220228,
 51222,
 50811,
 51237,
 815,
 1530,
 227467,
 51465,
 50883,
 803,
 225651,
 50885,
 1538,
 848,
 225690,
 50884,
 786,
 1522,
 3746,
 51029,
 50893,
 225625,
 50808,
 816,
 225667,
 3766,
 791,
 1525,
 220615,
 50912,
 50841,
 50931,
 807,
 811,
 1529,
 50809,
 3745,
 225664,
 220621,
 226537,
 818,
 225668,
 1531,
 50960,
 51088,
 51265,
 828,
 227457,
 51240,
 829,
 1535,
 227464,
 50971,
 50822,
 837,
 1536,
 220645,
 226534,
 50983,
 50824,
 51007,
 51105,
 51010,
 50973,
 50867,
 50956,
 825,
 1533,
 227466,
 51275,
 813,
 220545,
 51221,
 50810,
 51279,
 833,
 50862,
 772,
 1521,
 227456,
 821,
 1532,
 220635]

In [60]:
def change_col_to_datetime(inputevents_mv, feature):
    inputevents_mv[feature] = pd.to_datetime(inputevents_mv[feature])
    return inputevents_mv


admits = pd.read_csv(os.path.join(data, "mimiciii/1.4/raw", "ADMISSIONS.csv.gz"))
inputevents_mv = pd.read_csv(os.path.join(data, "mimiciii/1.4/raw", "INPUTEVENTS_MV.csv.gz"))

### Merge medication and admission data
inputevents_mv = pd.merge(inputevents_mv, admits, how="inner", on=["HADM_ID", "SUBJECT_ID"])
inputevents_mv = change_col_to_datetime(inputevents_mv, 'ADMITTIME')
inputevents_mv = change_col_to_datetime(inputevents_mv, 'ENDTIME')
inputevents_mv = change_col_to_datetime(inputevents_mv, 'STARTTIME')
inputevents_mv['MedTimeFromAdmit'] = inputevents_mv['ENDTIME']-inputevents_mv['ADMITTIME']

### Add medication information from D_ITEMS table in MIMIC III dataset (like label name)
med_data = pd.read_csv(os.path.join(data, "mimiciii/1.4/raw", "D_ITEMS.csv.gz"))
med_data = med_data[med_data["DBSOURCE"]=="metavision"]

# Merge medication data with medication labels
m_p_df = pd.merge(inputevents_mv, med_data, how="inner", on="ITEMID")

In [ ]:
# MIMIC
mimic_parser = mimic.MIMICParser(data=data, res=res, gender=gender, age_b=age_b, age_a=age_a, ethnicity=ethnicity, lab_mapping=lab_mapping)
m_med1, m_med2, m_labs = mimic_parser.parse()
## Querier
mimic_data_querier = querier.DatasetQuerier(
    data = data,
    res = res,
    gender=gender, 
    age_b=age_b, 
    age_a=age_a, 
    ethnicity=ethnicity, 
    lab_mapping=lab_mapping
)
m_final_lab_med_data = mimic_data_querier.generate_med_lab_data(m_labs, m_med1, m_med2, before_windows, after_windows)
## Discovery
discovery.ClinicalDiscoveryAnalysis()
## Plots
plotter = plots.ClinicalPlotAnalysis(
    data = data,
    res = res,
    gender=gender, 
    age_b=age_b, 
    age_a=age_a, 
    ethnicity=ethnicity, 
    lab_mapping=lab_mapping
)
m_corrs_data_df = plotter.plot(m_final_lab_med_data, m_labs, before_windows=before_windows, after_windows=after_windows)

In [ ]:
# HIRID
hirid_mapping = constants.HIRID_MAPPING
hirid_parser = hirid.HiRiDParser(data=raw_path, res=res_path, gender=gender, age_b=age_b, age_a=age_a)
h_med1, h_med2, h_labs = hirid_parser.parse()
lab_ids = [l for k in hirid_mapping.values() for l in k]
h_labs_1 = h_labs[h_labs.OldITEMID.isin(lab_ids)]

hirid_data_querier = querier.DatasetQuerier(
    data = raw_path,
    res = res_path,
    gender=gender, 
    age_b=age_b, 
    age_a=age_a, 
    ethnicity=ethnicity, 
)
final_h_final_lab_med_data, raw_h_final_lab_med_data = hirid_data_querier.generate_med_lab_data(h_labs_1, h_med1, h_med2, before_windows, after_windows)

h_plotter = plots.ClinicalPlotAnalysis(
    data = raw_path,
    res = res_path,
    gender=gender, 
    age_b=age_b, 
    age_a=age_a, 
    ethnicity="", 
    lab_mapping={}
)
h_corrs_data_df = h_plotter.plot(final_h_final_lab_med_data, h_labs, before_windows=before_windows, after_windows=after_windows)